In [1]:
import os
os.chdir("../")
!pwd

/home/aditya/network_security


In [171]:
from pathlib import Path
from pydantic import BaseModel,HttpUrl

class ModelTrainerConfig(BaseModel):
    ## from config
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
  
    mlflow_uri: HttpUrl
    mlflow_experiment: str
    standard_scaler_name: str
    
    ## from params
    models: list
    hyperparams: dict

    ## from schema 
    target_column: str

In [176]:
from src.NetworkSecurity.constants import *
from src.NetworkSecurity.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self)->ModelTrainerConfig:

        config = self.config.model_trainer
        params = self.params.model_trainer
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            mlflow_uri = config.mlflow_uri,
            mlflow_experiment = config.mlflow_experiment,
            standard_scaler_name = config.standard_scaler_name,

            models = list(params.model.keys()),
            hyperparams = params.model.to_dict(),
            
            target_column = schema.name           
        )

        return model_trainer_config

In [177]:
cm = ConfigurationManager()
model_trainer_config = cm.get_model_trainer_config()

[2025-03-31 14:16:37,546: INFO: common : Yaml File: config/config.yaml loaded successfully]
[2025-03-31 14:16:37,548: INFO: common : Yaml File: params.yaml loaded successfully]
[2025-03-31 14:16:37,551: INFO: common : Yaml File: schema.yaml loaded successfully]
[2025-03-31 14:16:37,551: INFO: common : created directory at: artifacts]
[2025-03-31 14:16:37,552: INFO: common : created directory at: artifacts/model_trainer]


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from src.NetworkSecurity.logging.logger import logger
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
import mlflow
from sklearn.preprocessing import StandardScaler
import pickle
from urllib.parse import urlparse
from mlflow.models import infer_signature
from sklearn.model_selection import GridSearchCV
from dotenv import load_dotenv
import datetime


load_dotenv()

class ModelTrainer:
    def __init__(self,config: ModelTrainerConfig):
        self.config = config
        os.environ["MLFLOW_TRACKING_USERNAME"] = "ADITYAAV80" 
        os.environ["MLFLOW_TRACKING_PASSWORD"] = "7c7d5f1b5994f5230784a07f1f9573168c40c053"
    
    def train(self):

        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        # Split features & target
        x_train = train_data.drop(columns=[self.config.target_column], axis=1)
        y_train = train_data[self.config.target_column]
        x_test = test_data.drop(columns=[self.config.target_column], axis=1)
        y_test = test_data[self.config.target_column]

        model_mapping = {
            "LogisticRegression": LogisticRegression(),
            "DecisionTreeClassifier": DecisionTreeClassifier(),
            "GradientBoostingClassifier": GradientBoostingClassifier(),
            "AdaBoostClassifier": AdaBoostClassifier(),
            "KNeighborsClassifier": KNeighborsClassifier(),
            "SVC": SVC(),
            "XGBClassifier": XGBClassifier(),
        }

        # Standardize features
        ss = StandardScaler()
        x_train = ss.fit_transform(x_train)

        # Standardize test features
        x_test = ss.transform(x_test)

        scaler_path = os.path.join(self.config.root_dir, self.config.standard_scaler_name)
        with open(scaler_path, "wb") as f:
            pickle.dump(ss, f)
            

        mlflow.set_tracking_uri(self.config.mlflow_tracking_uri)  # For tracking runs
        mlflow.set_registry_uri(self.config.mlflow_registry_uri)  # For model registry

        mlflow.set_experiment(self.config.mlflow_experiment)  # Set the experiment


        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        signature = infer_signature(x_train, y_train)

        models = self.config.models
        hyperparams = self.config.hyperparams

        best_model = None
        best_score = -float("inf")

        for model_name in models:
            model = model_mapping[model_name]
            params = hyperparams.get(model_name, {})

            grid_search = GridSearchCV(model, param_grid=params, cv=5, scoring="accuracy", n_jobs=-1)
            grid_search.fit(x_train, y_train)

            for param, score in zip(grid_search.cv_results_["params"], grid_search.cv_results_["mean_test_score"]):
                with mlflow.start_run():
                    print(f"{param}: {score:.4f}")

                    mlflow.log_params(param)  
                    mlflow.log_metric("train_accuracy", f"{score:.4f}")
                    mlflow.log_param("model_name", model_name)

                    # Track best model based on training accuracy (since GridSearchCV only works on train)
                    if score > best_score:  
                        best_score = score  
                        best_model = grid_search.best_estimator_  
                        best_param = param 
                        best_model_name = model_name 

        #  Now test the best model on the test set
        test_acc = best_model.score(x_test, y_test)


        
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        model_filename = f"{self.config.root_dir}/{best_model_name}_{timestamp}.pkl"

        # Save the best model
        with open(model_filename, "wb") as f:
            pickle.dump(best_model, f)

        # Save metadata (best parameters)
        metadata_filename = f"{self.config.root_dir}/{best_model_name}_{timestamp}_metadata.txt"
        with open(metadata_filename, "w") as f:
            f.write(f"Model: {best_model_name}\n")
            f.write(f"Best Params: {best_param}\n")
            f.write(f"Train Accuracy: {best_score}\n")

        # Log the best model based on test accuracy
        with mlflow.start_run():
            mlflow.log_params(best_param)
            mlflow.log_metric("test_accuracy", f"{test_acc:.4f}")  # Now test accuracy is logged
            mlflow.log_param("model_name",best_model_name)

            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(best_model, "model", registered_model_name="Best Model", signature=signature)
            else:
                mlflow.sklearn.log_model(best_model, "model", signature=signature)



In [179]:
mt = ModelTrainer(model_trainer_config)
mt.train()

https://dagshub.com/adityaav80/E2E-Network-Security.mlflow


/home/aditya/miniconda3/envs/network_security/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/aditya/miniconda3/envs/network_security/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/aditya/miniconda3/envs/network_security/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/aditya/miniconda3/envs/network_security/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/aditya/miniconda3/envs/network_security/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which 

{'C': 0.01, 'penalty': 'l1', 'solver': 'liblinear'}: 0.9264
🏃 View run orderly-mouse-686 at: https://dagshub.com/adityaav80/E2E-Network-Security.mlflow/#/experiments/3/runs/54d9a6a20a69446f936dab9cc40c52f0
🧪 View experiment at: https://dagshub.com/adityaav80/E2E-Network-Security.mlflow/#/experiments/3
{'C': 0.01, 'penalty': 'l1', 'solver': 'saga'}: 0.9275
🏃 View run efficient-pig-698 at: https://dagshub.com/adityaav80/E2E-Network-Security.mlflow/#/experiments/3/runs/c2925af9d3474a02858b88114ee11641
🧪 View experiment at: https://dagshub.com/adityaav80/E2E-Network-Security.mlflow/#/experiments/3
{'C': 0.01, 'penalty': 'l2', 'solver': 'liblinear'}: 0.9355
🏃 View run aged-cod-970 at: https://dagshub.com/adityaav80/E2E-Network-Security.mlflow/#/experiments/3/runs/fc95aedf9de04939abf5411daf7c751d
🧪 View experiment at: https://dagshub.com/adityaav80/E2E-Network-Security.mlflow/#/experiments/3
{'C': 0.01, 'penalty': 'l2', 'solver': 'saga'}: 0.9355
🏃 View run orderly-cub-305 at: https://dagshub

Registered model 'Best Model' already exists. Creating a new version of this model...
2025/03/31 14:17:58 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Best Model, version 4
Created version '4' of model 'Best Model'.


🏃 View run classy-ape-920 at: https://dagshub.com/adityaav80/E2E-Network-Security.mlflow/#/experiments/3/runs/fce5d8a75a0a41ac81dcbd73d6917a4f
🧪 View experiment at: https://dagshub.com/adityaav80/E2E-Network-Security.mlflow/#/experiments/3
